In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(5, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.fc2(x)
        return F.tanh(x)


model = MyModel()

out = model(torch.rand(5))
print(out)
print(out.shape)
print(model(torch.rand(5).unsqueeze(0)).shape)
print(model(torch.rand(5).unsqueeze(0).unsqueeze(0)).shape)

tensor([-0.2849], grad_fn=<TanhBackward0>)
torch.Size([1])
torch.Size([1, 1])
torch.Size([1, 1, 1])


In [3]:
out.shape

torch.Size([1])

In [4]:
print(model.fc1.state_dict())


OrderedDict([('weight', tensor([[ 0.4266,  0.1418,  0.0787, -0.0603, -0.1385],
        [ 0.0955,  0.0340, -0.1283, -0.4389,  0.3964],
        [-0.0377, -0.1160,  0.4302, -0.0834,  0.0234],
        [ 0.2476, -0.1388,  0.1002, -0.2152, -0.4105],
        [-0.0242,  0.1318,  0.2198,  0.1554, -0.1337],
        [-0.0094, -0.1013,  0.1563,  0.2894, -0.1974],
        [-0.3503, -0.0909, -0.0662,  0.2203,  0.0793],
        [-0.1848,  0.0303, -0.3050, -0.2767, -0.2853],
        [-0.0500,  0.3723, -0.3137,  0.1515, -0.3475],
        [ 0.2134, -0.0008,  0.0760,  0.1344, -0.1029]])), ('bias', tensor([ 0.2230,  0.3677, -0.1096, -0.0972,  0.3266, -0.3441, -0.0099, -0.1012,
        -0.4028,  0.2908]))])


In [5]:
torch.tensor(2).shape

torch.Size([])

In [6]:
isinstance(model.fc1, nn.Module)

True

In [7]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
writer.add_graph(model, torch.rand(5))
writer.close()

In [8]:
nn.Tanh?

Init signature: nn.Tanh(*args, **kwargs) -> None
Docstring:     
Applies the Hyperbolic Tangent (Tanh) function element-wise.

Tanh is defined as:

.. math::
    \text{Tanh}(x) = \tanh(x) = \frac{\exp(x) - \exp(-x)} {\exp(x) + \exp(-x)}

Shape:
    - Input: :math:`(*)`, where :math:`*` means any number of dimensions.
    - Output: :math:`(*)`, same shape as the input.

.. image:: ../scripts/activation_images/Tanh.png

Examples::

    >>> m = nn.Tanh()
    >>> input = torch.randn(2)
    >>> output = m(input)
Init docstring: Initialize internal Module state, shared by both nn.Module and ScriptModule.
File:           ~/projects/passing_the_torch/venv/lib/python3.10/site-packages/torch/nn/modules/activation.py
Type:           type
Subclasses:     

In [9]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(5, 10)
        self.nl1 = nn.Tanh()
        self.fc2 = nn.Linear(10, 1)
        self.nl2 = nn.Tanh()

    def forward(self, x):
        x = self.fc1(x)
        x = self.nl1(x)
        x = self.fc2(x)
        return self.nl2(x)


model = MyModel()

out = model(torch.rand(5))
print(out)
print(out.shape)
print(model(torch.rand(5).unsqueeze(0)).shape)
print(model(torch.rand(5).unsqueeze(0).unsqueeze(0)).shape)

tensor([0.1259], grad_fn=<TanhBackward0>)
torch.Size([1])
torch.Size([1, 1])
torch.Size([1, 1, 1])


In [10]:
writer = SummaryWriter()
writer.add_graph(model, torch.rand(5))
writer.close()

In [12]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2207,  0.1466, -0.3568, -0.2860, -0.0585],
         [-0.1582,  0.1217, -0.0859,  0.1306,  0.1662],
         [-0.0158,  0.4182, -0.3181,  0.0978, -0.1387],
         [ 0.1375,  0.3619,  0.2441, -0.3241, -0.0193],
         [-0.1960,  0.4269, -0.2684,  0.0177,  0.4463],
         [-0.1160, -0.0506, -0.2181,  0.1954,  0.2758],
         [-0.3748,  0.1646, -0.1075, -0.1940, -0.4468],
         [ 0.3785,  0.1593, -0.3964, -0.0366, -0.2333],
         [ 0.1707, -0.0325, -0.0823,  0.0911, -0.3582],
         [ 0.1998, -0.1256,  0.2642, -0.2502,  0.2250]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0451,  0.3004, -0.0041,  0.4164, -0.0115, -0.3451, -0.3801, -0.1956,
         -0.1690,  0.3703], requires_grad=True),
 Parameter containing:
 tensor([[ 0.1877, -0.3158,  0.0916,  0.1542,  0.1127,  0.0930,  0.1966,  0.2425,
           0.0706,  0.2424]], requires_grad=True),
 Parameter containing:
 tensor([0.2331], requires_grad=True)]

In [14]:
class MyModel(nn.Module):
    def __init__(self, num_layers=3):
        super(MyModel, self).__init__()
        self.layers = []
        in_features = 5
        out_features = 10
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(in_features, out_features))
            in_features = out_features   
        self.layers.append(nn.Linear(in_features, 1))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            x = F.tanh(x)
        return x


model = MyModel()

In [15]:
model.state_dict()

OrderedDict()

In [16]:
model(torch.rand(5))

tensor([-0.3304], grad_fn=<TanhBackward0>)

In [18]:
class MyModel(nn.Module):
    def __init__(self, num_layers=3):
        super(MyModel, self).__init__()
        layers = []
        in_features = 5
        out_features = 10
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(in_features, out_features))
            in_features = out_features   
        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            x = F.tanh(x)
        return x


model = MyModel(2)
model.state_dict()

OrderedDict([('layers.0.weight',
              tensor([[ 0.3956, -0.3880,  0.4201, -0.1354, -0.1536],
                      [ 0.1374,  0.0503,  0.0612,  0.0231,  0.1588],
                      [-0.0798, -0.2296, -0.0735,  0.3649,  0.3362],
                      [ 0.2273,  0.3117,  0.1105, -0.0899,  0.3984],
                      [-0.2068, -0.1243, -0.0746, -0.0251,  0.0485],
                      [-0.1613, -0.2822, -0.2108, -0.1591,  0.0344],
                      [ 0.0259, -0.3709, -0.3758, -0.1356,  0.3477],
                      [-0.3030,  0.0275, -0.3157,  0.1007,  0.2047],
                      [-0.0222,  0.0705,  0.2005, -0.3808, -0.0279],
                      [ 0.0557,  0.2277,  0.3824, -0.3172,  0.1456]])),
             ('layers.0.bias',
              tensor([ 0.2578, -0.0806,  0.1190,  0.2998,  0.2751,  0.1743,  0.2498, -0.1855,
                      -0.3150,  0.1348])),
             ('layers.1.weight',
              tensor([[ 0.0866, -0.0983, -0.1061, -0.1938,  0.2220, -0.04

In [20]:
class MyModel(nn.Module):
    def __init__(self, num_layers=3):
        super(MyModel, self).__init__()
        self.layers = nn.Sequential()
        in_features = 5
        out_features = 10
        for _ in range(num_layers - 1):
            self.layers.extend([nn.Linear(in_features, out_features), nn.Tanh()])
            in_features = out_features   
        self.layers.extend([nn.Linear(in_features, 1), nn.Tanh()])
        

    def forward(self, x):
        return self.layers(x)


model = MyModel()

In [21]:
writer = SummaryWriter()
writer.add_graph(model, torch.rand(5))
writer.close()